In [127]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json
import pickle
import glob
import tqdm

pd.set_option("max.columns", 131)

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix
from scipy import sparse
import joblib as jb
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/cesar/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
df = pd.read_csv("raw_data_all_labeled2.csv", index_col=0).dropna(subset=["y"])
df.shape

(1410, 15)

In [9]:
df = df.loc[0:715]

In [42]:
df.head(10)

,uploader,title,y,upload_date,user,view_count,like_count,dislike_count,thumbnail,width,height,categories,tags,channel_url,description
0,Yanjun Qi,S0-Introduction-Module3: Deep Learning and AI ...,0.0,2020-08-25,UCHMYETgeGbNHVHLidZSV8BQ,22,NaN,NaN,https://i.ytimg.com/vi/LkPmTGw1jqo/hqdefault.j...,1280,672,Science & Technology,Machine Learning,http://www.youtube.com/channel/UCHMYETgeGbNHVH...,Course Web: \nhttps://qiyanjun.github.io/2020f...
1,Ciência dos Dados,Machine Learning no Ensino Médio,0.0,2020-08-25,UCd3ThZLzVDDnKSZMsbK0icg,3,NaN,NaN,https://i.ytimg.com/vi_webp/R_gBq8IfwJc/maxres...,1920,1080,Education,machine learning|data science,http://www.youtube.com/channel/UCd3ThZLzVDDnKS...,"A matemática, sempre ela....\n\nDe uma maneira..."
2,iKennyHD,NBA LIVE 22: EA COULD USE DEEP MACHINE LEARNIN...,0.0,2020-08-25,KennyCallOfDuty,47,NaN,NaN,https://i.ytimg.com/vi/Tix2xon9MSs/maxresdefau...,1920,1080,Gaming,iKennyHD|nba live20|nba live 20|nba 2k20|live2...,http://www.youtube.com/channel/UCGMtoj9V9Go_im...,Wanna Donate? paypal.me/iKennyYT is where you ...
3,Amazon Web Services,Amazon Aurora Machine Learning – SageMaker Int...,0.0,2020-08-25,AmazonWebServices,335,NaN,NaN,https://i.ytimg.com/vi/w-2ip78NxAw/maxresdefau...,1920,1080,Science & Technology,AWS|Amazon Web Services|Cloud|AWS Cloud|Cloud ...,http://www.youtube.com/channel/UCd6MoB9NC6uYN2...,Learn how you can turn relational data into in...
4,"GMRIT, Rajam, AP",Machine Learning and Deep Learning Implementat...,1.0,2020-08-25,UC8g7hz4oXFzXNryt8h1gRPw,1486,NaN,NaN,https://i.ytimg.com/vi/f6XIY_M7FlA/hqdefault.j...,1280,720,People & Blogs,NaN,http://www.youtube.com/channel/UC8g7hz4oXFzXNr...,Resource Person\nMr.S.Aravinth Seshadri\nCerti...
5,Pranshi Verma,Machine Learning unit_5 (SOLUTION) revision cl...,0.0,2020-08-25,UCKymBWR3K8nQ1_trc-qyCHg,55,NaN,NaN,https://i.ytimg.com/vi_webp/B2YupcaJcWw/maxres...,1920,1080,Education,Genetic Algorithms: an illustrative example|Hy...,http://www.youtube.com/channel/UCKymBWR3K8nQ1_...,PDF LINK : ) https://drive.google.com/file/d/1...
6,Amaze Lab,Fifty New Alien Worlds Just Discovered with a ...,0.0,2020-08-25,UCKXEESjIEsKmGzws3fDKwFA,54,NaN,NaN,https://i.ytimg.com/vi_webp/mcsibAr8ZDI/maxres...,1920,1080,Science & Technology,Amazelab|science|space|tech|space tech|space e...,http://www.youtube.com/channel/UCKXEESjIEsKmGz...,"For the first time, astronomers have used an a..."
7,Unacademy Computer Science,Supervised Machine Learning : Linear Regressio...,1.0,2020-08-25,UCFWCFYvqnAMT-jcCqTp_SlA,68,NaN,NaN,https://i.ytimg.com/vi/57fBQ8Xiim8/maxresdefau...,854,480,Education,machine learning|machine learning gate exam|ma...,http://www.youtube.com/channel/UCFWCFYvqnAMT-j...,"In this live lecture, you will learn the Machi..."
8,e-tube KTU by jasmin,Machine learning/lecture 8,0.0,2020-08-25,UCp2slhMRlZ-w85K6QREKpAg,23,NaN,NaN,https://i.ytimg.com/vi_webp/IX6k7IndFVw/maxres...,1280,720,People & Blogs,machine learning|ML|Cs467|computer science|ktu,http://www.youtube.com/channel/UCp2slhMRlZ-w85...,Ktu CS467
9,Pythonist,Matrices in Machine Learning | Machine Learnin...,0.0,2020-08-25,UCGbcVi5c9XlAHkqKcHY3lLw,107,NaN,NaN,https://i.ytimg.com/vi_webp/ErMlaB5Bgjk/maxres...,1920,1080,Science & Technology,python|learn python|django|efficient python|py...,http://www.youtube.com/channel/UCGbcVi5c9XlAHk...,#machine_learning #matrices #linear_algebra #d...


In [108]:
vectorizer = TfidfVectorizer(stop_words='english')

In [126]:
title_bow_train_desc = vectorizer.fit_transform(df['description'])
title_bow_train_title = vectorizer.fit_transform(df['title'])

In [123]:
df_teste = pd.DataFrame(title_bow_train.toarray(), columns=vectorizer.vocabulary_)

In [128]:
teste_bow = sparse.hstack([title_bow_train_desc, title_bow_train_title])

In [124]:
df_teste

,description,title
0,1.0,0.0
1,0.0,1.0


In [101]:
corpus_index = [n[:10] for n in df['description']]

In [102]:
df_teste  = pd.DataFrame(title_bow_train.T.todense(),index= vectorizer.get_feature_names(), columns= corpus_index )

In [103]:
df_teste.head()

,Course Web,A matemáti,Wanna Dona,Learn how,Resource P,PDF LINK :,For the fi,In this li,Ktu CS467,#machine_l,Regression,This is th,Learn how,in this tu,The object,Formulatin,Data da vi,Reference,"Thursday,",This Keras,Histograms,Welcome to,Machine Le,This is th,Hello ever,Stacking E,Pantech Gr,This video,PLAYLIST:,Day-2: Mac,Google mac,National l,Derrick J,UPSC FEVER,The automa,Traditiona,Deep Java,Original F,This is a,"Olá, tudo",This is th,A walkthro,Classify H,Voodoo is,In this li,E aí pesso,Nesse víde,What is Ma,In this vi,Ronald van,Intro to M,Azure Mach,🔥 Edureka,This video,🔥Get the c,In this vi,HR Analysi,Day-1 Mach,Enjoy the,Hello Frie,Enjoy the,Cluj-Napoc,Join my te,This video,Enjoy the,...,In this vi,► Cómo esc,Ken Jee is,Meet Nick,FREE Data,Don't Just,In this 28,In this vi,Do you wan,"""In this t",Florida is,Yacine spe,Prof. Madh,Neste víde,In this vi,"""In this L",Subscribe,Manuela em,🔥Edureka P,Conheça al,Neste vide,IIT Madras,In this vi,IBF on Dem,Realistic,Enjoy the,Hands on S,Please joi,Data scien,New corona,Join us fo,It is no l,Want some,GitHub let,Through a,🔥Intellipa,This video,Whether yo,Prof. Dr.,Data Scien,How To Bec,Acompanhe,MIT’s Josh,If you wor,🤓 Link to,github lin,In this vi,Website: w,HBR named,✨ Best Dat,Data Scien,Eastern Un,Nesse víde,One of the,#DataDrive,В этом вид,The Amazon,🔥Edureka D,Learn Data,Statistics,What is th,[Inscriçõe,Acompanhe,This cours,►Checkout
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037199,0.256705,0.0,0.0,0.0,0.0,0.0,0.0,0.008373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.020881,0.020698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044094,0.0,0.0,0.0,0.0,0.0,0.018778,0.0,0.0,0.0,0.0,0.0,0.0,0.041985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.110907,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.042472,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [41]:
df_teste[df_teste['Regression'] > 0][['Regression']].sort_values(by="Regression", ascending=False)

,Regression
regression types,0.620333
types ml,0.571841
types,0.339169
regression,0.312270
ml,0.275021


In [44]:
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
df.loc[711]

uploader                                              Sadhna Singh
title            What Is The Data Science? | Data Science Under...
y                                                                0
upload_date                                             2020-06-04
user                                      UCMlXsTrmAAe4GqNl7Imbcqw
view_count                                                     632
like_count                                                     NaN
dislike_count                                                  NaN
thumbnail        https://i.ytimg.com/vi/tXJzP5H4DP4/maxresdefau...
width                                                         1280
height                                                         720
categories                                               Education
tags             Sadhna Singh|Data science|Data scientist|Data ...
channel_url      http://www.youtube.com/channel/UCMlXsTrmAAe4Gq...
description      What is the data science ?\nData science role

In [63]:
text = df.loc[[711]]['description']


711    What is the data science ?\nData science role ...
Name: description, dtype: object

In [55]:
vectorizer = CountVectorizer()

In [64]:
#tokenize and build vocab
vectorizer.fit(text)

CountVectorizer()

In [65]:
print(vectorizer.vocabulary_)

{'what': 11, 'is': 4, 'the': 10, 'data': 2, 'science': 9, 'role': 8, 'and': 0, 'responsibilities': 7, 'machine': 6, 'learning': 5, 'artificial': 1, 'intelligence': 3}


In [68]:
vector = vectorizer.transform(text)

In [69]:
#summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

(1, 12)
<class 'scipy.sparse.csr.csr_matrix'>
[[1 1 2 1 3 1 1 1 1 2 2 3]]


In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
text = ["The quick brown fox jumped over the lazy dog.", "The dog.", "The fox"]

In [89]:
vectorizer = TfidfVectorizer()

In [90]:
vectorizer.fit(text)

TfidfVectorizer()

In [91]:
print(vectorizer.vocabulary_)

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}


In [92]:
print(vectorizer.idf_)

[1.69314718 1.28768207 1.28768207 1.69314718 1.69314718 1.69314718
 1.69314718 1.        ]


In [96]:
vector = vectorizer.transform([text[0]])

In [97]:
print(vector.shape)

(1, 8)


In [98]:
print(vector.toarray())

[[0.36388646 0.27674503 0.27674503 0.36388646 0.36388646 0.36388646
  0.36388646 0.42983441]]
